# Visualization

### Import Libraries

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
from sklearn import preprocessing

##### Variable Initialize

In [ ]:
#Table Name
table_name =resultantTableName
result_df = pd.DataFrame()

##### Fetching the Resultant Data from Postfresql

In [ ]:
try:
    # Create instance
    engine   = postgresEngine

    #Reading the data from Postgresql
    result_df = pd.read_sql("select * from \""+table_name+"\" ",engine);

except Exception as e:
    print('Error:\n',e)

##### Air Quality Index based on Continents

## Which continent has the worst Air Quality Index based on the countries in the continent with a population density of more than 150,000?

In [ ]:
sns.set(style='dark')
fig, axs = plt.subplots(ncols=6)
#Querying the records based on the continents for the visualization
Europe = result_df.query("continent_name == 'Europe'")
Asia = result_df.query("continent_name == 'Asia'")
North_America = result_df.query("continent_name == 'North America'")
Africa = result_df.query('continent_name == "Africa"')
South_America = result_df.query('continent_name == "South America"')
oceania = result_df.query('continent_name == "Oceania"')

#Visualization based on the Continents for the Air Quality and Countries.
sns.barplot(x="Air_Quality_Index", y="country_name", data=Europe, ci=100,ax =axs[0])
sns.barplot(x="Air_Quality_Index", y="country_name", data=Asia, ci=100,ax =axs[1])
sns.barplot(x="Air_Quality_Index", y="country_name", data=North_America, ci=100,ax =axs[2])
sns.barplot(x="Air_Quality_Index", y="country_name", data=South_America, ci=100,ax =axs[3])
sns.barplot(x="Air_Quality_Index", y="country_name", data=oceania, ci=100,ax =axs[4])
sns.barplot(x="Air_Quality_Index", y="country_name", data=Africa, ci=100,ax =axs[5])

fig = plt.gcf()
fig.set_size_inches(50, 30)

plt.suptitle('Continents based Air Quality Index',fontsize=25, size=25)
plt.subplots_adjust(top=.925)
plt.show()


## Which gases play a significant role in the degradation of the air quality of a place?

In [ ]:
factors=result_df.loc[:,('Air_Quality_Index','Carbon_Monoxide','Nitrogen_Monoxide','Nitrogen_dioxide','Sulphur_dioxide','Ozone','Fine_Particles_matter','Coarse_particulate_Matter','Ammonia')]
#Normalizing the Data in the range 0 to 1
scaler = preprocessing.MinMaxScaler()
names = factors.columns
factors.fit= scaler.fit_transform(factors)
scaledfactors_df = pd.DataFrame(factors.fit, columns=names)
#Cluster heatmap for Correlation between the Pollutants
graphPlot = sns.clustermap(scaledfactors_df.corr(),cmap="YlGnBu")
graphPlot.fig.suptitle("HeatMap For Pollutants", fontsize =20)
graphPlot.fig.tight_layout()
graphPlot.fig.subplots_adjust(top=0.95)


## What are the average levels of most toxic gases for each of the continents

In [ ]:
continents = result_df.loc[:,("continent_name",'country_name','Carbon_Monoxide','Nitrogen_dioxide','Nitrogen_Monoxide','Sulphur_dioxide')]
continents=continents.groupby(['continent_name']).mean().reset_index()
# Set up the matplotlib figure
f, (ax1, ax2, ax3,ax4) = plt.subplots(4, 1, figsize=(10, 10), sharex=True)
plt.title("Pollutant levels based on Continents", fontsize =20, y=5)

# Carbon Monoxide
sns.barplot(x="continent_name", y="Carbon_Monoxide", palette="rocket", ax=ax1,data=continents)
ax1.axhline(0, color="k", clip_on=False)
ax1.set_ylabel("Carbon_Monoxide")

# Sulphur_dioxide
sns.barplot(x="continent_name", y="Sulphur_dioxide", palette="vlag", ax=ax2,data=continents)
ax2.axhline(0, color="k", clip_on=False)
ax2.set_ylabel("Sulphur_dioxide")

# Nitrogen Monoxide
sns.barplot(x="continent_name", y="Nitrogen_Monoxide", palette="deep", ax=ax3,data=continents)
ax3.axhline(0, color="k", clip_on=False)
ax3.set_ylabel("Nitrogen_Monoxide")

# Nitrogen dioxide
sns.barplot(x="continent_name", y="Nitrogen_dioxide", palette="deep", ax=ax4,data=continents)
ax4.axhline(0, color="k", clip_on=False)
ax4.set_ylabel("Nitrogen_dioxide")

# Finalize the plot
sns.despine(bottom=True)
plt.setp(f.axes, yticks=[])
# plt.tight_layout(h_pad=2)
plt.tight_layout()
           

## Weather Type Co-relation Across Fine Particles Matter vs Coarse Particulate_Matter vs Humidity vs Pressure

In [ ]:
newDf = result_df[['Fine_Particles_matter','Coarse_particulate_Matter','pressure', 'weather_title']]
graphPlot = sns.pairplot(newDf, hue='weather_title', palette='inferno')
graphPlot.fig.suptitle("Weather Type Co-relation Across Fine Particles Matter vs Coarse Particulate_Matter vs Humidity vs Pressure")
# graphPlot.fig.tight_layout()
graphPlot.fig.subplots_adjust(top=0.95)

## Does high windspeed increase the density of fine particles?

In [ ]:
graphPlot=sns.jointplot(x="Fine_Particles_matter", y="wind_speed", kind="kde", color="#4CB391",data=result_df)
graphPlot.fig.suptitle("Correlation between Fine_Particles_matter & wind_speed", fontsize =20)
graphPlot.fig.tight_layout()
graphPlot.fig.subplots_adjust(top=0.90)